In [1]:
!pip install tensorflow

In [6]:
!pip install catboost

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib

In [10]:
train_dir = r'C:\Users\shams\Desktop\Sections\Computer vision\Alzheimer_s Dataset\Alzheimer_s Dataset\train'
test_dir = r'C:\Users\shams\Desktop\Sections\Computer vision\Alzheimer_s Dataset\Alzheimer_s Dataset\test'

In [12]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [16]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [18]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 4220 images belonging to 4 classes.


In [20]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 1279 images belonging to 4 classes.


In [22]:
# Define the model architecture up to the feature extraction layer
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(64),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\shams\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\shams\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


132/132 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step


In [26]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

40/40 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step


In [28]:
# Use CatBoost classifier
cb_classifier = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6)
cb_classifier.fit(train_features, train_labels)

0:	learn: 1.2963977	total: 167ms	remaining: 2m 47s
1:	learn: 1.2279886	total: 189ms	remaining: 1m 34s
2:	learn: 1.1726620	total: 209ms	remaining: 1m 9s
3:	learn: 1.1194178	total: 229ms	remaining: 57s
4:	learn: 1.0744756	total: 249ms	remaining: 49.6s
5:	learn: 1.0365639	total: 270ms	remaining: 44.8s
6:	learn: 1.0087588	total: 291ms	remaining: 41.2s
7:	learn: 0.9804177	total: 310ms	remaining: 38.4s
8:	learn: 0.9563202	total: 329ms	remaining: 36.3s
9:	learn: 0.9325721	total: 349ms	remaining: 34.5s
10:	learn: 0.9103930	total: 367ms	remaining: 33s
11:	learn: 0.8931484	total: 386ms	remaining: 31.8s
12:	learn: 0.8772205	total: 405ms	remaining: 30.8s
13:	learn: 0.8623723	total: 423ms	remaining: 29.8s
14:	learn: 0.8508703	total: 443ms	remaining: 29.1s
15:	learn: 0.8381257	total: 462ms	remaining: 28.4s
16:	learn: 0.8268040	total: 480ms	remaining: 27.7s
17:	learn: 0.8151305	total: 498ms	remaining: 27.2s
18:	learn: 0.8042393	total: 517ms	remaining: 26.7s
19:	learn: 0.7951210	total: 537ms	remaining

In [29]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)

In [32]:
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 58.01%


In [34]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor6.h5')

In [36]:
# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier.pkl')

['catboost_classifier.pkl']

In [38]:
from keras.models import load_model
import joblib
import numpy as np
from tensorflow.keras.preprocessing import image
from catboost import CatBoostClassifier

# Load the saved models
cnn_model = load_model('cnn_feature_extractor6.h5')
catboost_classifier = joblib.load('catboost_classifier.pkl')  # Load CatBoost model

def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Ensure prediction is an integer
    predicted_class = int(prediction[0])

    # Interpret the prediction
    class_labels = {0: 'MildDemented', 1: 'ModerateDemented', 2: 'NonDemented', 3: 'VeryMildDemented'}  # Adjust based on your dataset labels
    result = class_labels[predicted_class]
    return result

# Test the function with a new image
img_path = r'C:\Users\shams\Desktop\Sections\Computer vision\Alzheimer_s Dataset\Alzheimer_s Dataset\test\NonDemented\26 (62).jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
The image is classified as: NonDemented


C:\Users\shams\AppData\Local\Temp\ipykernel_20980\3561859418.py:26: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predicted_class = int(prediction[0])
